# model

> data structures for clue

In [ ]:
#| default_exp model

In [ ]:
#| hide
#from nbdev.showdoc import *

In [2]:
#| export
from random import shuffle, randint, sample
from collections import namedtuple

In [3]:
#| export
people = []
people.append(["Miss Scarlet","red",u"\U0001f460","Scar."])
people.append(["Colonel Mustard","yellow",u"\U0001F9D0","Must."])
people.append([" Mrs. White","white",u"\U0001F97C","White"])
people.append(["Mr. Green","green",u"\U0001F409","Green"])
people.append(["Professor Plum","magenta",u"\U0001F346","Plum "])
people.append(["Mrs. Peacock","blue",u"\U0001F99A","Peac."])


In [4]:
#| export
weapons = []
weapons.append(["Dagger","cyan",u"\U0001F52A","Knife"])
weapons.append(["Revolver","cyan",u"\U0001F52B","Gun"])
weapons.append(["Rope","cyan",'🪢',"Rope"])
weapons.append(["Wrench","cyan",'🔧',"Wrench"])
weapons.append(["Candlestick","cyan",'🕯️',"Candle"])
weapons.append(["Lead Pipe","cyan",'🪠',"Pipe"])

In [5]:
#| export
rooms = []
rooms.append(["Ball Room","cyan","","Ball"])
rooms.append(["Billiard Room","cyan","","Bill"])
rooms.append(["Conservatory","cyan","","Consv"])
rooms.append(["Dining Room","cyan","","Dining"])
rooms.append(["Hall","cyan","","Hall"])
rooms.append(["Kitchen","cyan","","Kitch"])
rooms.append(["Lounge","cyan","","Lounge"])
rooms.append(["Library","cyan","","Lib"])
rooms.append(["Study","cyan","","Study"])

In [6]:
#| export
GameState = namedtuple('GameState', ['player', 'game', 'moves', 'adjacency','board'])

### Accessors on the data
We have both the two value version of an object and the single int version. The two is easier to generate. The one is good for keys

#| export
rooms = []
rooms.append(["Ball Room","grey","","Ball"])
rooms.append(["Billiard Room","grey","","Bill"])
rooms.append(["Conservatory","grey","","Consv"])
rooms.append(["Dining Room","grey","","Dining"])
rooms.append(["Hall","grey","","Hall"])
rooms.append(["Kitchen","grey","","Kitch"])
rooms.append(["Lounge","grey","","Lounge"])
rooms.append(["Library","grey","","Lib"])
rooms.append(["Study","grey","","Study"])

In [7]:
#| export


def cardLookup(kind,index):
    if kind == 1:
        return people[index]
    if kind == 2:
        return weapons[index]
    return rooms[index]

def keyCard(kind,index):
    return kind*100 + index

def keyForArray(v):
    return keyCard( v[0] , v[1])
    
def cardForArray(v):
    return cardLookup(v[0],v[1])
    
def keyLookUp(key):
    kind = key // 100
    index = key % 100
   # print(key,kind,index)
    return cardLookup(kind,index)

In [8]:
#| hide
keyForArray([1,3]),cardForArray([1,3])

(103, ['Mr. Green', 'green', '🐉', 'Green'])

In [ ]:
#| hide


In [ ]:
### Board details

In [9]:
#| export

def createClue(playerCount):
    bhand = [[1,randint(0,len(people)-1)],[2,randint(0,len(weapons)-1)],[3,randint(0,len(rooms)-1)]]
    ahand = [keyCard(x[0],x[1]) for x in bhand]
    ploc = sample(range(len(rooms)),len(people))
    
    person = ahand[0]
    
    pdeck = [keyCard(1,x) for x in range(len(people))]
    deck = [[1,x] for x in range(len(people)) if x != person]
    deck = [x for x in pdeck if x != person]
    
    weapon = ahand[1]
    wdeck = [keyCard(2,x) for x in range(len(weapons))]
    deck += [x for x in wdeck if x != weapon]
   
    room = ahand[2]
    wdeck = [keyCard(3,x) for x in range(len(rooms))]
    deck += [x for x in wdeck if x != room]

    
    shuffle(deck)
    decks = [ahand]
    
    for i in range(playerCount):
        decks.append([])
        
   

    cIndex = 1
    for card in deck:
        decks[cIndex].append(card)
        cIndex += 1
        if cIndex > playerCount:
            cIndex = 1
    decks.append(ploc)        
    return decks



        
     

In [10]:
#| hide
acount = 5
aState =  GameState(1,createClue(acount),[],[set([x-1,x+1]) for x in range(len(rooms))],[])
aState.game 

[[100, 203, 302],
 [308, 105, 102, 101],
 [103, 301, 303, 205],
 [300, 304, 201, 307],
 [306, 204, 104],
 [200, 305, 202],
 [3, 5, 1, 8, 6, 2]]

In [11]:
#| export
def playerIndex(playerID):
    return playerID - 1
    
def roomOf(pc,state):
    ploc = state.game[-1]
    return ploc[playerIndex(pc)]

def handOf(state):
    return state.game[state.player]
    
def roomsAround(x,state):
    return state.adjacency[x]
    
    l = len(rooms)
    r1 = x -1
    if r1 < 0:
        r1 = l - 1
    r3 = x + 1
    if r3 > l - 1:
        r3 = 0
    return [r1,x,r3]


In [12]:
#| hide
roomOf(1,aState), handOf(aState),roomsAround(roomOf(1,aState),aState)

(3, [308, 105, 102, 101], {2, 4})

In [14]:
#| export
def relocatePlayers(playerID , suspectID,room,state):

    state.game[-1][playerIndex(playerID)] = room
    state.game[-1][suspectID] = room
    return  GameState(state.player,state.game,state.moves,state.adjacency,state.board)
    

In [15]:
#| hide
aState = relocatePlayers(1,3,6,aState)
aState.game[-1]

[6, 5, 1, 6, 6, 2]

## Flow

In [16]:
#| export
def keysForGuess(guess):
    p,w,l = guess[:3]
    h = [[1,p],[2,w],[3,l]]
    h = [keyCard(x[0],x[1]) for x in h]
    return h

def responseToGuess(guess,asker,answered,shown):
    p,w,l = guess
    return (p,w,l,asker, answered,shown)

In [17]:
#| hide
aguess = (0,2,2)
aresponse =  responseToGuess(aguess,1,4, keysForGuess(aguess)[0])
keysForGuess(aguess), aresponse, keysForGuess(aresponse)

([100, 202, 302], (0, 2, 2, 1, 4, 100), [100, 202, 302])

In [18]:
#| export
def getAnswer(state,guess):

    h = keysForGuess(guess)
    shuffle(h)

    playCount = len(state.game) - 1
    
    c = state.player+1
    
    if c > playCount:
        c = 1
    while c != state.player:
        hand = state.game[c]
        for y in h:
            for x in hand:

                if x == y :
                    return responseToGuess(guess,state.player,c,y)
        c = c+1
        if c >= playCount:
            c = 1
    return (-1,0,0)

In [19]:
#| hide
resp = getAnswer(aState,aguess)
aState.moves.append(resp)
resp,aState.moves

((0, 2, 2, 1, 5, 202), [(0, 2, 2, 1, 5, 202)])

In [20]:
#| export


def getSeen(state):
    foo = [x[-1] for x in state.moves if x[3] == state.player]
    return handOf(state) + foo


def resetPlayer(state):
    return GameState(1, state.game, state.moves,state.adjacency,state.board)

def bumpPlayer(state):
    i = state.player + 1
    if i > len(state.game) - 2:
        return resetPlayer(state)
    return GameState(i,state.game, state.moves,state.adjacency,state.board)
    

In [21]:
#| hide
aState = resetPlayer(aState)
for i in range(len(aState.game)-2):
    print(getSeen(aState))
    aState = bumpPlayer(aState)

[308, 105, 102, 101, 202]
[103, 301, 303, 205]
[300, 304, 201, 307]
[306, 204, 104]
[200, 305, 202]


In [22]:
#| export
def passedOnResponse(response,state):
    playMax = len(state.game) - 1
    s = response[-3] + 1
    end = response[-2]
    ret = []
    if s > playMax:
        s = 1
    while s != end:
        ret.append(s-1)
        s = s + 1
        if s > playMax:
            s = 1
    return ret

In [ ]:
#| export
def notInHands(state):
    ret = dict()
    
    for x in state.moves:
        
        
        thePasses = passedOnResponse(x,state) 
        
        if len(thePasses) > 0:
            shown = x[-1]
            if shown in ret:
                addToMe = ret[shown]
            else:
                addToMe = set()
            for aPass in thePasses:
                addToMe.add(aPass)
            ret[shown] = addToMe
                
       
    return ret

In [ ]:
#| export
def hasGuess(guess,moves):
    p, w, l = guess[:3]
    c = (p,w,l)
    for x in moves:
        p, w, l = x[:3]
        xc = (p,w,l)
        if c == xc:
            return True
    return False

In [ ]:
#| hide
hasGuess(aguess,aState.moves),hasGuess((-1,-1,-1),aState.moves)

(True, False)